In [19]:
import os,sys,shutil
from pathlib import Path

#https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FDT/UserGuide#BEDPOSTX
#Create starting bedpostx_start directory with data.nii.gz nodif_brain_mask.nii.gz bvecs bvals

def bedpostx(subjdir, sesdir):
    sourcedir = Path(sesdir,'dti')
    fslprocdir = Path(sourcedir, 'fsl')
    fslprocdir.mkdir(exist_ok=True)
    bedpostxdir = Path(fslprocdir, "bedpostx")
    bedpostxdir.mkdir(exist_ok=True)
    subjroot = "_".join([subjdir.name, sesdir.name])
    dtimif = sourcedir / (subjroot + '_ppd.mif')
    dtimaskmif = sourcedir / (subjroot + '_mask_ppd.mif')
    sourcedwinii = fslprocdir / (subjroot + "_ppd.nii")
    sourcedwimasknii = fslprocdir / (subjroot + "_mask_ppd.nii")
    sourcebvecs = fslprocdir / (subjroot + ".bvec")
    sourcebvals = fslprocdir / (subjroot + '.bval')
    !mrconvert -force -fslgrad $sourcebvecs $sourcebvals $dtimif $sourcedwinii
    !mrconvert -force $dtimaskmif $sourcedwimasknii
    bedpostxbasename = Path(bedpostxdir, subjroot)
    bedpostxsubj_dir = bedpostxdir / subjroot
    bedpostxsubj_dir.mkdir(exist_ok=True)
    bedpostxstartdwiniigz = bedpostxsubj_dir / 'data.nii.gz'
    bedpostxstartmaskniigz = bedpostxsubj_dir / 'nodif_brain_mask.nii.gz'
    bedpostxstartbvecs = bedpostxsubj_dir / 'bvecs'
    bedpostxstartbvals = bedpostxsubj_dir / 'bvals'
    shutil.copy(sourcedwinii, bedpostxstartdwiniigz)
    shutil.copy(sourcedwimasknii, bedpostxstartmaskniigz)
    shutil.copy(sourcebvecs, bedpostxstartbvecs)
    shutil.copy(sourcebvals, bedpostxstartbvals)
    os.chdir(bedpostxsubj_dir)
    !bedpostx_gpu $bedpostxsubj_dir --model=1
    
    

In [ ]:
scratch_proc = Path('/scratch/jdrussell3/bids_proc')

subjdirs = (subjdir for subjdir in scratch_proc.iterdir() if subjdir.is_dir())

for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sesdirs:
        bedpostx(subjdir, sesdir)
        

mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] copying from "/scratch/j.../dti/sub-001_ses-01_ppd.mif" to "/scratch/j.../fsl/sub-001_ses-01_ppd.nii"
mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [WARNING] requested datatype (Bit) not supported - substituting with UInt8
mrconvert: [100%] copying from "/scratch/j...sub-001_ses-01_mask_ppd.mif" to "/scratch/j...sub-001_ses-01_mask_ppd.nii"
---------------------------------------------
------------ BedpostX GPU Version -----------
---------------------------------------------
subjectdir is /scratch/jdrussell3/bids_proc/sub-001/ses-01/dti/fsl/bedpostx/sub-001_ses-01
/scratch/jdrussell3/bids_proc/sub-001/ses-01/dti/fsl/bedpostx/sub-001_ses-01 has already been processed: /scratch/jdrussell3/bids_proc/sub-001/ses-01/dti/fsl/bedpostx/sub-001_ses-01.bedpostX.
Delete or rename /scratch/jdrussell3/bids_proc/sub-001/ses-01/dti/fsl/bedpostx/sub-001_ses-01.bedpostX before repeating the pro